In [77]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [78]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['PMT_Background_275Deg_071217.txt', 'PMT_Signal_275Deg_071217.txt', 'PMT_Signal_280Deg_071217.txt', 'PMT_Background_280Deg_071217.txt', 'PMT_Background_270Deg_071217.txt', 'PMT_Signal_270Deg_071217.txt', 'PMT_Signal_265Deg_071217.txt', 'PMT_Background_265Deg_071217.txt', 'PMT_Background_260Deg_071217.txt', 'PMT_Signal_260Deg_071217.txt', 'PMT_Signal_255Deg_071217.txt', 'PMT_Background_255Deg_071217.txt', 'PMT_Background_250Deg_071217.txt', 'PMT_Signal_250Deg_071217.txt', 'PMT_Signal_245Deg_071217.txt', 'PMT_Background_245Deg_071217.txt']


In [79]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [80]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [87]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,100):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(100,len(allFiles))
PDvalues = PDvalues.values.reshape(100,len(allFiles))
Background = np.array([PDvalues[:,0],PDvalues[:,3],PDvalues[:,4],PDvalues[:,7],PDvalues[:,8],PDvalues[:,11],PDvalues[:,12],PDvalues[:,15]])
Background_mean = np.mean(Background, axis = 1)
PMT_Raw = np.array([PDvalues[:,1],PDvalues[:,2],PDvalues[:,5],PDvalues[:,6],PDvalues[:,9],PDvalues[:,10],PDvalues[:,13],PDvalues[:,14]])
MeanPMT = np.mean(PMT_Raw,axis = 1)
PMT_STD = np.std(PMT_Raw,axis = 1)
PMT_Err = PMT_STD/(100**0.5)
w = (PMT_Err/(MeanPMT*log(10)))**-2
PMT_Signal = np.absolute(MeanPMT-Background_Mean)
THzEnergy = np.array([480,450,390,310,235,160,105])
LogPMT = np.log10(PMT_Signal)
LogTHz = np.log10(THzEnergy)



print(Background_Mean)
print(MeanPMT)
print(MeanPMT-Background_Mean)

[-0.172841 -0.174479 -0.168453 -0.161327 -0.157979 -0.153124 -0.151708
 -0.153002]
[-0.207742 -0.215507 -0.203582 -0.190215 -0.17666  -0.172229 -0.161715
 -0.159599]
[-0.034901 -0.041028 -0.035129 -0.028888 -0.018681 -0.019105 -0.010007
 -0.006597]


In [88]:
Delta = sum(w)*sum(w*LogTHz**2) - (sum(w*LogTHz))**2
A = (sum(w*LogTHz**2)*sum(w*LogPMT) - sum(w*LogTHz)*sum(w*LogTHz*LogPMT))/Delta
B = (sum(w)*sum(w*LogTHz*LogPMT) - (sum(w*LogPMT)*sum(w*LogTHz)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogTHz)**2)**0.5)
errorA = (sum(w*LogTHz**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)

[ 1019653.40581481   854900.64858593  1177041.99879608   815068.74964978
  1411437.5114817   1491091.61027318   900345.40342529   941968.72429127]
-3.90188359923
0.938940146015
0.00315616366402
0.00132789331822


In [91]:
plt.plot(LogTHz, A + B*LogTHz,'r',label = 'Fit: y=0.94x -3.90')
plt.plot(LogTHz, LogPMT,'b.', label = 'Data')
plt.grid(True)
plt.xlabel('Log(THz Energy)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('LogPMTVsLogTHz_071217.png')